In [27]:
from selenium import webdriver
driver = webdriver.Chrome()

from selenium.webdriver.common.by import By
import time

data = []

for page in range(1, 11):
    url = f"https://www.saramin.co.kr/zf_user/jobs/list/job-category?cat_mcls=2&page={page}&page_count=100&sort=RD"
    driver.get(url)
    time.sleep(10)  # Adding a delay to let the page load

    box_item_list=driver.find_elements(By.CSS_SELECTOR, 'div.box_item')

    for box_item in box_item_list:
        temp = {}
        temp['company_name'] = box_item.find_element(By.CSS_SELECTOR, '.company_nm > .str_tit').text
        # temp['company_link'] = box_item.find_element(By.CSS_SELECTOR, '.company_nm > a').get_attribute('href')
        temp['is_headhunting'] = '헤드헌팅' in box_item.find_element(By.CSS_SELECTOR, ".company_nm").text
        temp['job_title'] = box_item.find_element(By.CSS_SELECTOR, '.notification_info a.str_tit').get_attribute('title')
        temp['job_link'] = box_item.find_element(By.CSS_SELECTOR, '.notification_info a.str_tit').get_attribute('href')
        temp['job_sectors'] = [i.text for i in box_item.find_elements(By.CSS_SELECTOR, '.notification_info .job_meta .job_sector span')]
        temp['work_place'] = box_item.find_element(By.CSS_SELECTOR, '.recruit_info .work_place').text
        temp['career_info'] = box_item.find_element(By.CSS_SELECTOR, '.recruit_info .career').text
        temp['education_info'] = box_item.find_element(By.CSS_SELECTOR, '.recruit_info .education').text
        temp['support_date'] = box_item.find_element(By.CSS_SELECTOR, '.support_info .date').text
        temp['deadline'] = box_item.find_element(By.CSS_SELECTOR, '.support_info .deadlines').text
        data.append(temp)

    print(page)


1
2
3
4
5
6
7
8
9
10


In [28]:
import pandas as pd
df = pd.DataFrame(data)
df.to_clipboard()
df.to_csv('saram.csv')


In [ ]:
driver.get(url)


In [ ]:
len(box_item_list)

In [23]:
box_item_list[0].find_element(By.CSS_SELECTOR, '.company_nm > .str_tit').text

'(주)커리어라이즈'

In [ ]:
driver.find_element(By.CSS_SELECTOR, '[data-mcls_cd_no="2"]').click()
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

wait = WebDriverWait(driver, 10)
button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#all_check_onedepth_2')))
button.click()


In [ ]:
#all_check_onedepth_2

In [ ]:
driver.find_element(
    By.CSS_SELECTOR, "#cphContents_cphContents_cphContents_ddlTeam"
).click()

In [ ]:

import time
import re
pattern = re.compile("playerId=([0-9]+)")
playid = []
select_team = "#cphContents_cphContents_cphContents_ddlTeam > option:nth-child({})"
select_page = "#cphContents_cphContents_cphContents_ucPager_btnNo{}"
for x in range(2,12):
    for_1 = select_team.format(x)
    driver.find_element(By.CSS_SELECTOR, for_1).click()
    time.sleep(2)
    #playid.extend(pattern.findall(driver.page_source))
    for y in range(1,6):
        f2 = select_page.format(y)
        try:
            driver.find_element(By.CSS_SELECTOR, f2).click()
            time.sleep(1)
            playid.extend(pattern.findall(driver.page_source))
        except Exception as e:
            print (e)
        time.sleep(2)


In [ ]:
driver.close()

# 개별 선수 정보 조회

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [ ]:
players = []

for id in tqdm(playid):
    url = f"https://www.koreabaseball.com/Record/Player/PitcherDetail/Basic.aspx?playerId={id}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    player_basic_infos = soup.find(class_="player_basic").find_all("span")
    edited_player_basic_infos = [info.text for info in player_basic_infos]
    edited_player_basic_infos.append(id)
    edited_player_basic_infos.append(soup.find(class_="team").text)
    players.append(edited_player_basic_infos)

In [ ]:
cols = [
    "Name",
    "BackNo",
    "Birthday",
    "Position",
    "HeightWeight",
    "Career",
    "Payment",
    "Salary",
    "Draft",
    "JoinInfo",
    "Id",
    "Team"
]

df = pd.DataFrame(players, columns=cols)
df

# 환산연봉

In [ ]:
def calculate_conversion(row):
    if row[2] == "달러":
        return int(row[1]) * 1311
    elif row[2] == "만원":
        return int(row[1]) * 10000
    else:
        return 0

df1 = df.Salary.str.split("(\d+)", expand=True)
df["환산연봉"] = df1.apply(calculate_conversion, axis=1)
df

# 파일로 출력

In [ ]:
for index, row in df.iterrows():
    id = row['Id']
    pd.DataFrame([row]).to_csv(f'./csv/player_info_{id}.csv', index=False, encoding='utf-8-sig')